In [1]:
using JuMP, Gurobi
using CSV, LinearAlgebra, DataFrames

In [2]:
#load the data and orgnize 
FL = repeat([Float64], inner=15)
dtype = append!([String], FL);

regional_EF = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/SC_regional/new_EF_SC.csv",header=1,delim=",", types=dtype) |> DataFrame    
capacity = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/SC_regional/capacity.csv",header=1,delim=",", types=dtype) |> DataFrame    
distance = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/SC_regional/distance.csv",header=1,delim=",") |> DataFrame 
LCA_model = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/SC_regional/LCA_model.csv",header=1,delim=",") |> DataFrame 
emi_sink = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/SC_regional/emission_sink.csv",header=1,delim=",") |> DataFrame;

In [9]:
cell_demand = 0.001*165*(1.369*1e6)*3          # annual demand of Li battery for tesla (2M EV/yr, 3 NMC111 pack/EV, 164.98 kg/pack)

global_sink = 1.099e10                        # global pub (ocean) CO2 sequestration (ton/yr)
global_sink_tot = 2.236e10                  # global total (ocean+land) CO2 sequestration (ton/yr)
global_emi = 3.53e10                          # global CO2 emission (ton/yr)
es_ratio = global_sink/global_emi
es_ratio_tot = global_sink_tot/global_emi
emission_c = emi_sink[!, "emission"]          # national CO2 emission (ton/yr)
sink_c = emi_sink[!, "sink ton/yr"]           # national CO2 sink (ton/yr)

EF_trans = 1.005/10000                        # ton CO2/km*ton (The average freight truck in the U.S. emits 161.8 grams of CO2 per ton-mile)
process = LCA_model[!,"process"]
countries = capacity[!,"Country"]
ncty = size(countries,1)                          # No. of countries
nproc = size(process,1);                          # No. of processes 

mkt_loc = findfirst(isequal("United States"), countries)
mkt_proc = findfirst(isequal("battery"), process)

# seperate model
cathode = 1:4
cell = 5:10
noncell = [12,13]
battery = [11,14]
scaler = LCA_model[!,"scaler"];

In [4]:
LCA_model

Row,process,input (material/kg battery),scaler
,String15,Float64,Float64
1,Li,0.11,0.383
2,Co,0.154,0.536
3,Mn,0.15,0.522
4,Ni,0.154,0.535
5,cathode,0.287,0.396
6,graphite,0.16,0.221
7,PP,0.0116,0.016
8,PE,0.0029,0.004
9,Cu,0.133,0.095


In [5]:
up_cath = scaler[1:4] * scaler[5] * scaler[11]
up_cell = scaler[5:10] * scaler[11]
cell_sef = scaler[11]
up_noncell = scaler[12:13] * scaler[14]
noncell_sef = scaler[14]
battery_sef = scaler[15];

In [6]:
vcat(up_cath, up_cell, cell_sef, up_noncell, noncell_sef, battery_sef)

15-element Vector{Float64}:
 0.1099593
 0.1538856
 0.1498662
 0.1535985
 0.2871
 0.160225
 0.0116
 0.0029
 0.06887499999999999
 0.1334
 0.725
 0.005
 0.004
 1.0
 1.0

In [10]:
cell_demand

677655.0

In [11]:
3*164.98/1000*1.36*10^6   

673118.4

In [12]:
vcat(up_cath, up_cell, cell_sef, up_noncell, noncell_sef, battery_sef) .* cell_demand

15-element Vector{Float64}:
  74514.4694415
 104281.34626800001
 101557.579761
 104086.7915175
 194554.75050000002
 108577.272375
   7860.798
   1965.1995
  46673.488124999996
  90399.177
 491299.875
   3388.275
   2710.62
 677655.0
 677655.0

In [13]:
input_amount = []
input_amount = vcat(up_cath, up_cell, cell_sef, up_noncell, noncell_sef, battery_sef) .* cell_demand
input_amount    # unit: ton

15-element Vector{Float64}:
  74514.4694415
 104281.34626800001
 101557.579761
 104086.7915175
 194554.75050000002
 108577.272375
   7860.798
   1965.1995
  46673.488124999996
  90399.177
 491299.875
   3388.275
   2710.62
 677655.0
 677655.0

----

In [14]:
model = Model(Gurobi.Optimizer);

Academic license - for non-commercial use only - expires 2023-11-27


In [15]:
#variables 
@variable(model, x[1:ncty, 1:nproc] >= 0)            # x[i,k] production amount of product k at location i
@variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0);   # y[i,j,k] ship product k from i to j 

In [16]:
# capacity constraint: x[i,k] <= capacity[i][k]
for k in 1:nproc
    for i in 1:ncty
        @constraint(model, x[i,k] <= capacity[!, 2:end][i,k])
    end
end

In [17]:
# node output flow constraint
for k in 1:nproc
    for i in 1:ncty
        @constraint(model, sum(y[i,j,k] for j in 1:ncty) == x[i,k])
    end
end

In [18]:
# node output flow to market y[i,38, 15]
for i in 1:ncty
    for j in [e for e in 1:ncty if e != mkt_loc]
        @constraint(model, y[i,j,mkt_proc] == 0)
    end
end

In [19]:
for k in 1:nproc-1
    @constraint(model, sum(x[i,k] for i in 1:ncty) == input_amount[k])
end

In [20]:
# @constraint(model, sum(x[i,9] for i in 1:ncty) == input_amount[9])

In [21]:
# # final demand constraint
# @constraint(model, sum(x[i, mkt_proc] for i in 1:ncty) == cell_demand);

In [22]:
# final demand constraint
@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) == cell_demand);

In [23]:
# cathode LCA constraints (index=5)
for k in cathode
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,5] * scaler[k])
    end
end


# cell LCA constraints (index=11)
for k in cell
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,11] * scaler[k])
    end
end


# non cell LCA constraints (index=14)
for k in noncell
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,14] * scaler[k])
    end
end


# battery LCA constraints (index=15)
for k in battery
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,15] * scaler[k])
    end
end


In [24]:
# 1. production emission (TES)
val_x = zeros(ncty, nproc)

for k in 1:nproc
    for i in 1:ncty
        val_x[i,k] = regional_EF[i, k+1] * (1 - sink_c[i]/emission_c[i] - es_ratio)
    end
end

proc_emi = sum(x .* val_x);

In [25]:
# 2. transpotration emission (TES, w/ supply)
trans_emi = 0

for i in 1:ncty
    for j in 1:ncty
        amount = sum(y[i,j,k] for k in 1:nproc)
        arc_emi = amount * distance[!, 2:end][i,j] * EF_trans
        arc_seq = arc_emi * (sink_c[j]/emission_c[j] + es_ratio)
        arc_net = arc_emi - arc_seq
        trans_emi += arc_net
    end
end

In [26]:
@objective(model, Min, proc_emi+trans_emi);

In [27]:
JuMP.optimize!(model)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 2103 rows, 13050 columns and 26912 nonzeros
Model fingerprint: 0x471a5f88
Coefficient statistics:
  Matrix range     [4e-03, 1e+00]
  Objective range  [2e-03, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+15]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 1801 rows and 11497 columns
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds
Infeasible or unbounded model

User-callback calls 23, time in user-callback 0.00 sec


In [82]:
optx = copy(JuMP.value.(x))
opty = copy(JuMP.value.(y));

In [83]:
optx

29×15 Matrix{Float64}:
  3010.8     0.0       0.0      0.0  …     0.0    0.0       0.0       0.0
     0.0  7400.0       0.0      0.0        0.0    0.0       0.0       0.0
  1800.0     0.0   33869.8  34713.3        0.0    0.0       0.0       0.0
  1540.0  4440.0       0.0      0.0        0.0    0.0       0.0       0.0
 18500.0     0.0       0.0      0.0        0.0  904.0       0.0       0.0
     0.0  2000.0       0.0      0.0  …  1130.0    0.0       0.0       0.0
     0.0     0.0       0.0      0.0        0.0    0.0       0.0       0.0
     0.0  1380.0       0.0      0.0        0.0    0.0       0.0       0.0
     0.0     0.0       0.0      0.0        0.0    0.0       0.0       0.0
     0.0     0.0       0.0      0.0        0.0    0.0       0.0       0.0
     0.0     0.0       0.0      0.0  …     0.0    0.0       0.0       0.0
     0.0  1200.0       0.0      0.0        0.0    0.0       0.0       0.0
     0.0     0.0       0.0      0.0        0.0    0.0       0.0       0.0
     ⋮         

In [84]:
using Tables
res = Tables.table(optx[1:end, 1:end]);
CSV.write("/Users/bourg/Desktop/x_matrix_regional.csv", res) 

"/Users/bourg/Desktop/x_matrix_regional.csv"

In [62]:
res_y02 = Tables.table(opty[1:end, 1:end, 02])
CSV.write("/Users/bourg/Desktop/y02_matrix.csv", res_y02) 

"/Users/bourg/Desktop/y02_matrix.csv"

In [38]:
# # 2. transpotration emission
# trans_emi = 0
# for k in 1:nproc
#     for i in 1:ncty
#         for j in 1:ncty
#             trans_emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans;
#         end
#     end
# end